In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
from xgboost import XGBClassifier, cv
import numpy as np
import pandas as pd

In [29]:
xgb = XGBClassifier()
xgb.cv()

AttributeError: 'XGBClassifier' object has no attribute 'cv'

In [23]:
MAX_WORDS = 10000

In [24]:
train_data = pd.read_csv('../Data/train.csv')

In [26]:
tfidf_vectorizer = TfidfVectorizer(max_features=MAX_WORDS)
tfidf_vectorizer.fit(train_data.text)

X_train = tfidf_vectorizer.transform(train_data.text)
y_train = train_data.label

In [20]:
EARLY_STOPPING_ROUNDS = 50
MAX_XGBOOST_ROUND = 1500
NUMBER_OF_CLASSES = 2

xgb_params = {'max_depth': 5,  # the maximum depth of each tree
              'eta': 0.2,  # the training step for each iteration
              'objective': 'multi:softmax',
              'num_class': NUMBER_OF_CLASSES,
              'min_child_weight': 2,
              'subsample': 1,
              'colsample_bytree': 0.5,
              'eval_metric': ['mlogloss', 'merror']}

k_fold = KFold(n_splits=2, random_state=1, shuffle=True)
result = cv(xgb_params, xgb.DMatrix(X_train, label=y_train), 
                num_boost_round=MAX_XGBOOST_ROUND, folds=k_fold,    
                early_stopping_rounds=EARLY_STOPPING_ROUNDS, verbose_eval=False)

0.00845	test-merror:0.11571+0.00493
[150]	train-mlogloss:0.14845+0.00213	train-merror:0.04229+0.00129	test-mlogloss:0.28432+0.00845	test-merror:0.11571+0.00517
[151]	train-mlogloss:0.14799+0.00223	train-merror:0.04194+0.00151	test-mlogloss:0.28418+0.00833	test-merror:0.11567+0.00513
[152]	train-mlogloss:0.14753+0.00226	train-merror:0.04179+0.00127	test-mlogloss:0.28404+0.00839	test-merror:0.11548+0.00471
[153]	train-mlogloss:0.14705+0.00221	train-merror:0.04148+0.00131	test-mlogloss:0.28397+0.00844	test-merror:0.11525+0.00481
[154]	train-mlogloss:0.14658+0.00225	train-merror:0.04127+0.00136	test-mlogloss:0.28384+0.00832	test-merror:0.11555+0.00469
[155]	train-mlogloss:0.14590+0.00208	train-merror:0.04105+0.00129	test-mlogloss:0.28377+0.00840	test-merror:0.11598+0.00441
[156]	train-mlogloss:0.14544+0.00202	train-merror:0.04070+0.00119	test-mlogloss:0.28374+0.00836	test-merror:0.11598+0.00446
[157]	train-mlogloss:0.14496+0.00204	train-merror:0.04047+0.00122	test-mlogloss:0.28379+0.00846	

In [21]:
print(result)

     train-mlogloss-mean  train-mlogloss-std  train-merror-mean  \
0               0.612051            0.000352           0.173967   
1               0.555668            0.002884           0.162005   
2               0.511843            0.001516           0.151710   
3               0.480213            0.002746           0.147931   
4               0.451717            0.002262           0.144403   
..                   ...                 ...                ...   
257             0.106322            0.001999           0.019191   
258             0.106018            0.001957           0.019076   
259             0.105669            0.002052           0.018787   
260             0.105324            0.001978           0.018526   
261             0.105014            0.001948           0.018439   

     train-merror-std  test-mlogloss-mean  test-mlogloss-std  \
0            0.001023            0.614373           0.000330   
1            0.002206            0.560457           0.002399   
2  

In [ ]:
xgb = XGBClassifier()

xgb.fit(X, y)

In [ ]:
test_data = pd.read_csv('../Data/test.csv')

X_test = tfidf_vectorizer.transform(test_data.text)
y_test = test_data.label

predicted = knn.predict(X_test)
print(f'Accuracy: {np.mean(predicted == y_test) * 100}%')